<a href="https://colab.research.google.com/github/vpysmennyi/machinelearning-learning/blob/main/Hometask7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision.datasets import STL10
from torch.utils.data import DataLoader
from torchvision.utils import save_image


class Print(nn.Module):
    def __init__(self):
        super(Print, self).__init__()

    def forward(self, x):
        print(x.shape)
        return x


class DAE(nn.Module):
    def __init__(self):
        super(DAE, self).__init__()

        encoder = nn.Sequential(
            # 1x96x96
            self.convBlock(3, 64, (3, 3), 1),
            nn.MaxPool2d(3, 3),
            self.convBlock(64, 64, (3, 3), 1),
            self.convBlock(64, 32, (3, 3), 1),
            self.convBlock(32, 16, (3, 3), 1),
            nn.MaxPool2d(4, 4),

            self.convBlock(16, 8, (3, 3), 1)

            # 8x8x8
        )

        decoder = nn.Sequential(
            self.deconvBlock(8, 16, (3, 3), 1),
            nn.UpsamplingBilinear2d(scale_factor=4),

            self.deconvBlock(16, 32, (3, 3), 1),
            self.deconvBlock(32, 64, (3, 3), 1),
            self.deconvBlock(64, 64, (3, 3), 1),

            nn.UpsamplingBilinear2d(scale_factor=3),
            self.deconvBlock(64, 3, (3, 3), 1)
        )

        self.net = nn.Sequential(encoder, decoder)

    def convBlock(self, input, output, kernel, padding):
        return nn.Sequential(
            nn.Conv2d(input, output, kernel_size=kernel, padding=padding),
            nn.ReLU(),
            nn.BatchNorm2d(output))

    def deconvBlock(self, input, output, kernel, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(input, output, kernel_size=kernel, padding=padding),
            nn.ReLU(),
            nn.BatchNorm2d(output))

    def forward(self, x):
        return self.net(x)


def calculate_mean_std(data):
    samples_mean, samples_std = [], []

    for i in range(len(data)):
        samples_mean.append(data[0][0].mean([1, 2]))
        samples_std.append(data[0][0].std([1, 2]))

    samples_mean = torch.stack(samples_mean).mean(0)
    samples_std = torch.stack(samples_std).mean(0)

    return samples_mean, samples_std


def makedir():
    dir = 'Images'
    if not os.path.exists(dir):
        os.makedirs(dir)


def save_decoded_image(img, name):
    img = img.view(img.size(0), 3, 96, 96)
    save_image(img, name)


batch_size = 16
lr = 1e-3
num_epochs = 10
noise_factor = 0.2

train_ds = STL10('.', split='train', folds=None, transform=transforms.ToTensor(), download=True)
X = torch.stack([train_ds[i][0] for i in range(len(train_ds))], 1).reshape(3, -1)
stl10_mean, stl10_std = X.mean(1), X.std(1)
print(stl10_mean, stl10_std)

transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize(stl10_mean, stl10_std),
                                ])

train_ds = STL10('.', split='unlabeled', folds=None, transform=transform, download=True)
val_ds = STL10('.', split='test', transform=transform, download=True)

train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=True)

makedir()
torch.cuda.init()

network = DAE()
opt = optim.Adam(network.parameters(), lr=lr)
loss_fn = nn.MSELoss()
train_loss = []

for epoch in range(num_epochs):
    epoch_loss = []

    network.train()
    for batch in train_dl:
        img, cl = batch

        noizy_img = img + noise_factor * torch.randn(img.shape)

        noizy_img = np.clip(noizy_img, 0, 1)
        opt.zero_grad()

        output = network(noizy_img)

        loss = loss_fn(output, noizy_img)

        loss.backward()
        opt.step()

        epoch_loss.append(loss.cpu().detach())

    epoch_loss = torch.stack(epoch_loss).mean()
    train_loss.append(epoch_loss)
    print(f' TRAIN | Epoch {epoch} - Loss: {epoch_loss:.6f}')
    save_decoded_image(noizy_img.cpu().data, './Images/noizy{}.png'.format(epoch))
    save_decoded_image(output.cpu().data, './Images/output{}.png'.format(epoch))

    torch.save(network.state_dict(), './drive/MyDrive/network.save')

network.eval()
for batch in val_dl:
    img, cl = batch

    noizy_img = img + noise_factor * torch.randn(img.shape)

    noizy_img = np.clip(noizy_img, 0, 1)

    output = network(noizy_img)

    loss = loss_fn(output, noizy_img)

    save_image(noizy_img, 'Images/testNoizyImage.png')
    save_image(output, 'Images/testOutputImage.png')
    break






Extracting ./stl10_binary.tar.gz to .
tensor([0.4467, 0.4398, 0.4066]) tensor([0.2603, 0.2566, 0.2713])
Files already downloaded and verified
Files already downloaded and verified
 TRAIN | Epoch 0 - Loss: 0.072605
 TRAIN | Epoch 1 - Loss: 0.041182
 TRAIN | Epoch 2 - Loss: 0.039829
 TRAIN | Epoch 3 - Loss: 0.039103
 TRAIN | Epoch 4 - Loss: 0.038668
 TRAIN | Epoch 5 - Loss: 0.038393


In [ ]:
# Pre-training
tensor([0.4467, 0.4398, 0.4066]) tensor([0.2603, 0.2566, 0.2713])
Files already downloaded and verified
Files already downloaded and verified
 TRAIN | Epoch 0 - Loss: 0.072605
 TRAIN | Epoch 1 - Loss: 0.041182
 TRAIN | Epoch 2 - Loss: 0.039829
 TRAIN | Epoch 3 - Loss: 0.039103
 TRAIN | Epoch 4 - Loss: 0.038668
 TRAIN | Epoch 5 - Loss: 0.038393

In [ ]:
import os
import torch
from datetime import datetime
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision.datasets import STL10
from torch.utils.data import DataLoader
from torchvision.utils import save_image


class Print(nn.Module):
    def __init__(self):
        super(Print, self).__init__()

    def forward(self, x):
        print(x.shape)
        return x


class Logger():
    def __init__(self, file = './CNN_execution.log'):
        self.logf = open(file, 'w')

    def write(self, text):
        self.logf.write(self.sysdate() + ' : ' + text + '\n')
        self.logf.flush()

    def close(self):
        self.logf.close()

    def sysdate(self):
        return datetime.now().strftime('%y%m%d %H:%M:%S')


class DAE(nn.Module):
    def __init__(self):
        super(DAE, self).__init__()

        encoder = nn.Sequential(
            # 1x96x96
            self.convBlock(3, 64, (3, 3), 1),
            nn.MaxPool2d(3, 3),
            self.convBlock(64, 64, (3, 3), 1),
            self.convBlock(64, 32, (3, 3), 1),
            self.convBlock(32, 16, (3, 3), 1),
            nn.MaxPool2d(4, 4),

            self.convBlock(16, 8, (3, 3), 1)

            # 8x8x8
        )

        decoder = nn.Sequential(
            self.deconvBlock(8, 16, (3, 3), 1),
            nn.UpsamplingBilinear2d(scale_factor=4),

            self.deconvBlock(16, 32, (3, 3), 1),
            self.deconvBlock(32, 64, (3, 3), 1),
            self.deconvBlock(64, 64, (3, 3), 1),

            nn.UpsamplingBilinear2d(scale_factor=3),
            self.deconvBlock(64, 3, (3, 3), 1)
        )

        self.net = nn.Sequential(encoder, decoder)

    def convBlock(self, input, output, kernel, padding):
        return nn.Sequential(
            nn.Conv2d(input, output, kernel_size=kernel, padding=padding),
            nn.ReLU(),
            nn.BatchNorm2d(output))

    def deconvBlock(self, input, output, kernel, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(input, output, kernel_size=kernel, padding=padding),
            nn.ReLU(),
            nn.BatchNorm2d(output))

    def forward(self, x):
        return self.net(x)


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        encoder = nn.Sequential(
            # 1x96x96
            self.convBlock(3, 64, (3, 3), 1),
            nn.MaxPool2d(3, 3),
            self.convBlock(64, 64, (3, 3), 1),
            self.convBlock(64, 32, (3, 3), 1),
            self.convBlock(32, 16, (3, 3), 1),
            nn.MaxPool2d(4, 4),

            self.convBlock(16, 8, (3, 3), 1)

            # 8x8x8
        )

        classifier = nn.Sequential(nn.Linear(512, 256),
                                   nn.ReLU(),
                                   nn.BatchNorm1d(256),
                                   nn.Linear(256,128),
                                   nn.ReLU(),
                                   nn.BatchNorm1d(128),
                                   nn.Linear(128,10)
                                   )

        self.net = nn.Sequential(encoder, nn.Flatten(), classifier)

    def convBlock(self, input, output, kernel, padding):
        return nn.Sequential(
            nn.Conv2d(input, output, kernel_size=kernel, padding=padding),
            nn.ReLU(),
            nn.BatchNorm2d(output))

    def deconvBlock(self, input, output, kernel, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(input, output, kernel_size=kernel, padding=padding),
            nn.ReLU(),
            nn.BatchNorm2d(output))

    def forward(self, x):
        return self.net(x)


batch_size = 16
lr = 1e-3
num_epochs = 40
noise_factor = 0.3

train_ds = STL10('.', split='train', folds=None, transform=transforms.ToTensor(), download=True)
X = torch.stack([train_ds[i][0] for i in range(len(train_ds))], 1).reshape(3, -1)
stl10_mean, stl10_std = X.mean(1), X.std(1)
print(stl10_mean, stl10_std)

transform = transforms.Compose([transforms.RandomCrop(size=(96,96), padding=3),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize(stl10_mean, stl10_std),
                                ])

val_transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(stl10_mean, stl10_std)
                                    ])

train_ds = STL10('.', split='train', folds=None, transform=transform, download=True)
val_ds = STL10('.', split='test', transform=val_transform, download=True)

train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=True)

log = Logger('./drive/MyDrive/CNN.log')

torch.cuda.init()

pre_train = DAE()
cl_network = CNN()
pre_train.load_state_dict(torch.load('./drive/MyDrive/network.save'))


loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(cl_network.parameters(), lr=lr)
for epoch in range(num_epochs):
    e_acc, e_loss = [], []
    val_acc, val_loss = [], []

    cl_network.train()
    for (X,Y) in train_dl:
        opt.zero_grad()

        Yp = None

        if epoch == 0:
            Yp = cl_network(pre_train(X))
        else:
            Yp = cl_network(X)
        loss = loss_fn(Yp, Y)

        loss.backward()
        opt.step()

        acc = (Yp.argmax(1) == Y).sum()/len(Y)

        e_loss.append(loss.detach().cpu())
        e_acc.append(acc)

    e_loss = torch.stack(e_loss).mean()
    e_acc = torch.stack(e_acc).mean()

    log.write(f' TRAIN | Epoch {epoch} - Loss: {e_loss:.6f} | Acc: {e_acc:.4f}')
    print(f' TRAIN | Epoch {epoch} - Loss: {e_loss:.6f} | Acc: {e_acc:.4f}')

    cl_network.eval()
    for (X, Y) in val_dl:
        Yp = cl_network(X)

        loss = loss_fn(Yp, Y)
        acc = (Yp.argmax(1) == Y).sum() / len(Y)

        val_acc.append(acc)
        val_loss.append(loss.detach().cpu())

    val_loss = torch.stack(val_loss).mean()
    val_acc = torch.stack(val_acc).mean()
    log.write(f' VAL   | Epoch {epoch} - Loss: {val_loss:.6f} | Acc: {val_acc:.4f}')
    print(f' VAL   | Epoch {epoch} - Loss: {val_loss:.6f} | Acc: {val_acc:.4f}')

log.close()

Extracting ./stl10_binary.tar.gz to .
tensor([0.4467, 0.4398, 0.4066]) tensor([0.2603, 0.2566, 0.2713])
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# CNN - NO pre-training
tensor([0.4467, 0.4398, 0.4066]) tensor([0.2603, 0.2566, 0.2713])
Files already downloaded and verified
Files already downloaded and verified
 TRAIN | Epoch 0 - Loss: 1.720096 | Acc: 0.3560
 VAL   | Epoch 0 - Loss: 1.429361 | Acc: 0.4784
 TRAIN | Epoch 1 - Loss: 1.407299 | Acc: 0.4856
 VAL   | Epoch 1 - Loss: 1.319004 | Acc: 0.5073
 TRAIN | Epoch 2 - Loss: 1.299600 | Acc: 0.5258
 VAL   | Epoch 2 - Loss: 1.251231 | Acc: 0.5412
 TRAIN | Epoch 3 - Loss: 1.218154 | Acc: 0.5583
 VAL   | Epoch 3 - Loss: 1.222840 | Acc: 0.5510
 TRAIN | Epoch 4 - Loss: 1.154979 | Acc: 0.5855
 VAL   | Epoch 4 - Loss: 1.125458 | Acc: 0.5881
 TRAIN | Epoch 5 - Loss: 1.101568 | Acc: 0.5966
 VAL   | Epoch 5 - Loss: 1.116038 | Acc: 0.5943
 TRAIN | Epoch 6 - Loss: 1.046154 | Acc: 0.6204